#  类-面向对象（获取沪宁、宁沪所有站点某日所有余票数据）

```
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
# station_ls_comb = []
for i in range(0,len(hu_ning)-1):
    ls = [hu_ning[i],hu_ning[i+1]]
#     station_ls_comb.append(ls)
    print ls
# print station_ls_comb
```

In [ ]:
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
ning_hu = ['NJH','ZJH','DYH','CZH','WXH','SZH','KNH','SHH']

## 1. 进行站点之间组合

# $C^{2}_8$   

In [ ]:
from itertools import combinations
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
com = combinations(hu_ning,2)
for i in com:
    print i

## 2. 抓取特定时间所有站点数据-运行一次

In [ ]:
import requests
import pandas as pd
import json
import arrow
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")

class HighSpeed(object):
    def __init__(self,date,from_station,to_station):
        self.date = date
        self.from_station = from_station    #发车站
        self.to_station = to_station        #到达站

    
#     url = 'https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT
#     &queryDate=2016-12-29&from_station=SHH&to_station=SZH'

    def get_pd(self):
        para = 'queryDate=' + self.date + '&from_station=' + self.from_station \
               +'&to_station=' + self.to_station
        r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                         para, verify=False).text
        return r

    def to_csv(self):
        js = self.get_pd()
        js2 = json.loads(js)
        data = js2['data']['datas']
        df = pd.DataFrame(data)
        df_use = df[[u'train_no',
                u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
                     ]]
        df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
        df_use[u'station_train_code'].str.startswith('D')]
        df_use.replace('--',0,inplace=True)
        df_use.replace(u'无',0,inplace=True)    
        
        tm = arrow.now().format('HH:mm')
        df_use['record_time'] = tm
        
#         df_use = df_choose[df_choose['record_time'] == df_choose['start_time']]
      
        df_use.to_csv('./data/' + self.date + 'TEST.csv',
                       mode = 'a', index = False, 
                      column = False,
                      encoding='utf-8')
         
if __name__ == '__main__':
    date = '2016-12-29'    
    hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
    com = combinations(hu_ning,2)
    for ls in com:
        hspeed = HighSpeed(date,ls[0],ls[1])
        hspeed.to_csv()
    print('finished')